In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0),  
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2, padding=0),
            nn.ELU(), 
            nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class PovTargetFusion(nn.Module):
    def __init__(self):
        super(PovTargetFusion, self).__init__()
        
        #self.visual_encoder = VisualEncoder()
        self.visual_dim = 512
        
        self.conv_1 = nn.Conv3d(7, 7, 1)
        self.linear_1 = nn.Linear(self.visual_dim, 7)
        #self.act_1 = nn.ELU()
        
        self.conv_2 = nn.Conv3d(7, 16, 3)
        self.linear_2 = nn.Linear(self.visual_dim, 16)
        self.act_2 = nn.ELU()

        self.conv_3 = nn.Conv3d(16, 32, 3)
        self.linear_3 = nn.Linear(self.visual_dim, 32)
        self.act_3 = nn.ELU()
        
        self.conv_4 = nn.Conv3d(32, 64, 3)
        self.linear_4 = nn.Linear(self.visual_dim, 64)
        self.act_4 = nn.ELU()
        
        self.conv_5 = nn.Conv3d(64, 128, 3)
        self.act_5 = nn.ELU()
        
        self.linear_6 = nn.Linear(128 * 9, 512)
        self.act_6 = nn.ELU()
        
    def forward(self, target, vis_features):
        Q = self.linear_1(vis_features).reshape(-1, 1, 7).permute(0, 2, 1)
        V = self.conv_1(target).reshape(-1, 9 * 11 * 11, 7)
        result = torch.sigmoid(torch.bmm(V, Q)) * V
        result = result.reshape(-1, 9, 11, 11, 7).permute(0, 4, 1, 2, 3)
        
        Q = self.linear_2(vis_features).reshape(-1, 1, 16).permute(0, 2, 1)
        V = self.conv_2(result).reshape(-1, 7 * 9 * 9, 16)
        result = torch.sigmoid(torch.bmm(V, Q)) * self.act_2(V)
        result = result.reshape(-1, 7, 9, 9, 16).permute(0, 4, 1, 2, 3)
        
        Q = self.linear_3(vis_features).reshape(-1, 1, 32).permute(0, 2, 1)
        V = self.conv_3(result).reshape(-1, 5 * 7 * 7, 32)
        result = torch.sigmoid(torch.bmm(V, Q)) * self.act_3(V)
        result = result.reshape(-1, 5, 7, 7, 32).permute(0, 4, 1, 2, 3)
        
        Q = self.linear_4(vis_features).reshape(-1, 1, 64).permute(0, 2, 1)
        V = self.conv_4(result).reshape(-1, 3 * 5 * 5, 64)
        result = torch.sigmoid(torch.bmm(V, Q)) * self.act_4(V)
        result = result.reshape(-1, 3, 5, 5, 64).permute(0, 4, 1, 2, 3)
        
        result = self.act_5(self.conv_5(result))
        result = torch.flatten(result, start_dim=1)
        result = self.act_6(self.linear_6(result))
        
        return result

In [4]:
from torch.nn.functional import one_hot

class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        visual_features_dim = 512
        target_features_dim = 9 * 11 * 11 
        self.visual_encoder = VisualEncoder()
        self.visual_encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AngelaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.target_encoder = PovTargetFusion()
        policy_hidden_dim = 256 
        self.policy_network = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ELU(),
            nn.Linear(512, policy_hidden_dim),
            nn.ELU(),
            nn.Linear(policy_hidden_dim, policy_hidden_dim),
            nn.ELU(),
        )
        self.action_head = nn.Linear(policy_hidden_dim, action_space.n)
        self.value_head = nn.Linear(policy_hidden_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.visual_encoder.cuda()
            self.target_encoder.cuda()
            self.policy_network.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs']
        pov = obs['pov'].permute(0, 3, 1, 2).float() / 255.0
        target = one_hot(obs['target_grid'].long(), num_classes=7).permute(0, 4, 1, 2, 3).float()
        if self.use_cuda:
            pov.cuda()
            target.cuda()
            
        with torch.no_grad():
            visual_features = self.visual_encoder(pov)
            
        target_features = self.target_encoder(target, visual_features)
        target_features = target_features.reshape(target_features.shape[0], -1)
        features = torch.cat([visual_features, target_features], dim=1)
        features = self.policy_network(features)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [5]:
visual_features_dim = 512
target_features_dim = 9 * 11 * 11
policy_hidden_dim = 256 

policy_network = nn.Sequential(
    nn.Linear(visual_features_dim + target_features_dim, 1024),
    nn.ELU(),
    nn.Linear(1024, 512),
    nn.ELU(),
    nn.Linear(512, policy_hidden_dim),
    nn.ELU(),
    nn.Linear(policy_hidden_dim, policy_hidden_dim),
    nn.ELU(),
    #nn.Linear(policy_hidden_dim, policy_hidden_dim),
    #nn.ELU(),
)

sum(p.numel() for p in policy_network.parameters())

2362368

In [6]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [7]:
class VisualObservationWrapper(ObsWrapper):
    def __init__(self, env, include_target=False):
        super().__init__(env)
        self.observation_space = {   
            'pov': gym.spaces.Box(low=0, high=255, shape=(64, 64, 3)),
            'inventory': gym.spaces.Box(low=0.0, high=20.0, shape=(6,)),
            'compass': gym.spaces.Box(low=-180.0, high=180.0, shape=(1,))
        }
        if include_target:
            self.observation_space['target_grid'] = \
                gym.spaces.Box(low=0, high=6, shape=(9, 11, 11))
        self.observation_space = gym.spaces.Dict(self.observation_space)

    def observation(self, obs, reward=None, done=None, info=None):
        if info is not None:
            if 'target_grid' in info:
                target_grid = info['target_grid']
                del info['target_grid']
            else:
                logger.error(f'info: {info}')
                if hasattr(self.unwrapped, 'should_reset'):
                    self.unwrapped.should_reset(True)
                target_grid = self.env.unwrapped.tasks.current.target_grid
        else:
            target_grid = self.env.unwrapped.tasks.current.target_grid
        return {
            'pov': obs['pov'].astype(np.float32),
            'inventory': obs['inventory'],
            'compass': np.array([obs['compass']['angle'].item()]),
            'target_grid': target_grid
        }

In [8]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

tasks = []
for i in range(1,156):
    if ('C'+str(i)) == 'C38': continue
    tasks.append('C'+str(i))
    
class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        if abs(rew) == 1:
            rew /= 10
            
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=250)
    env.update_taskset(TaskSet(preset=tasks))
    #env = PovOnlyWrapper(env)
    env = VisualObservationWrapper(env, include_target=True)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 3,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 5_000,
             "lr": 1e-4,
             #"gamma": 0.99,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO All Tasks pretrained (AngelaCNN+fusion) (3 noops after placement) r: -0.01 div10"
                  }
              }

        },
        loggers=[WandbLogger],
        local_dir="/IGLU-Minecraft/checkpoints/all_tasks",
        keep_checkpoints_num=50,
        checkpoint_freq=5,
        checkpoint_at_end=True)

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_ef690_00000,PENDING,


2021-11-07 15:39:46,527	INFO wandb.py:170 -- Already logged into W&B.
2021-11-07 15:39:46,549	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id 3ecf2_00000 but id ef690_00000 is set.
(pid=9113) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9113)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


(pid=9113) 2021-11-07 15:39:51,116	WARNING ppo.py:143 -- `train_batch_size` (5000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1666.
(pid=9113) 2021-11-07 15:39:51,117	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=9113) 2021-11-07 15:39:51,117	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=9108) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9108)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=9117) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to 

Result for PPO_my_env_ef690_00000:
  agent_timesteps_total: 9996
  custom_metrics: {}
  date: 2021-11-07_15-44-45
  done: false
  episode_len_mean: 103.87368421052632
  episode_media: {}
  episode_reward_max: 4.690000000000007
  episode_reward_mean: -0.7170526315789476
  episode_reward_min: -1.6000000000000008
  episodes_this_iter: 95
  episodes_total: 95
  experiment_id: 5986ad5d0d3246ff80964fd8aadc2f01
  hostname: cds2
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999998
          cur_lr: 0.00010000000000000002
          entropy: 2.881463285185333
          entropy_coeff: 0.01
          kl: 0.005726082684665919
          policy_loss: -0.014017798388615632
          total_loss: 0.024265516103587598
          vf_explained_var: -0.32391759753227234
          vf_loss: 0.06595273009425777
    num_agent_steps_sampled: 9996
    num_agent_steps_trained: 9996
    num_steps_sampled: 9996
    num_steps_trained: 9996
  iterations_since_res

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ef690_00000,RUNNING,10.55.229.87:9113,28,6453.43,279888,1.9893,12.12,-2.02,101.74


(pid=9115) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9115)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=9108) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9108)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for PPO_my_env_ef690_00000:
  agent_timesteps_total: 289884
  custom_metrics: {}
  date: 2021-11-07_17-31-49
  done: false
  episode_len_mean: 99.94
  episode_media: {}
  episode_reward_max: 8.360000000000017
  episode_reward_mean: 1.7643000000000049
  episode_reward_min: -2.0699999999999994
  episodes_this_iter: 100
  episodes_total: 2847
  experiment_id: 5986ad5d0d3246ff80964fd8aadc2f01
  hostname: cds2
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 1.0125
          cur_lr: 0.00010000000000000002
          entropy: 2.574681407162267
          entropy_coeff: 0.01
          kl: 0.0186158744570433
          policy_loss: -0.04669317620464115
          total_loss: 0.16800117368738238
          vf_explained_var: 0.780113697052002
          vf_loss: 0.22159259046435867
    num_agent_steps_sampled: 289884
    num_agent_steps_trained: 289884
    num_steps_sampled: 289884
    num_steps_trained: 289884
  iterations_since_restore: 29
  node_ip: 1

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_ef690_00000,RUNNING,10.55.229.87:9113,29,6699.27,289884,1.7643,8.36,-2.07,99.94


(pid=9117) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=9117)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
